# General utils

> Some general utility functions that are used throughout the package.

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from torch.utils.data import Dataset
from typing import Union, List, Tuple, Literal
from gymnasium.spaces import Space
from ddopnew.dataloaders.base import BaseDataLoader

import numpy as np

In [ ]:
#| export

def check_parameter_types(
                            *args,                      # any number of parameters to be checked
                            parameter_type=np.ndarray   # the expected type for each parameter
):
    
    """
    Checks if each argument in args is of the specified type, defaulting to np.ndarray.
    """
    
    for index, arg in enumerate(args):
        if not isinstance(arg, parameter_type):
            raise TypeError(f"Argument {index+1} of {len(args)} is of type {type(arg).__name__}, expected {parameter_type.__name__}")

Example usage for the `check_parameter_types` function.

In [ ]:

a = np.array([1, 2, 3])
b = [1, 2, 3]

try:
    check_parameter_types(a, b)
except TypeError as e:
    print(e)

Argument 2 of 2 is of type list, expected ndarray


In [ ]:
#| export
class Parameter():

    """
    Simple class to handle parameters in the environment. The advantage of this class is that it can be
    used to set parameters that may change over time and accessed by multiple objects such as the 
    environment, agent or dataloaders.
    """
    
    def __init__(self,
            value: int | float | list[int] | list[float] | np.ndarray,  # the value of the parameter  
            min_value: int | float | list[int] | list[float] | np.ndarray = None,  # the minimum value of the parameter
            max_value: int | float | list[int] | list[float] | np.ndarray = None,  # the maximum value of the parameter
            shape: tuple[int] = (1,)): # the shape of the parameter

        self._min_value = min_value
        self._max_value = max_value
        
        self.set_value(value, shape)

    def __call__(self):
        """
        Returns the parameter value. Alternative to get_value().

        """
        return self.get_value()

    def get_value(self):
        
        """
        Returns the parameter value.

        """

        return self._value

    def set_value(self, 
                    value: Union[int, float, List[int], List[float], np.ndarray], # the value of the parameter
                    shape: Tuple[int] = (1,)): # the shape of the parameter
       
        """
        Set the value of the parameter. The value can be a scalar, list or numpy array. The shape of the value
        must be the same as the shape of the parameter.

        All values, including scalars, are converted to numpy arrays. If the value is a scalar, it is reshaped
        to the shape of the parameter. If the value is a list or numpy array, it must have the same shape as the
        parameter. If the value is a scalar, it is reshaped to the shape of the parameter.

        Optionally, the value can be clipped to a minimum and maximum value.
        """

        if isinstance(value, (int, float)):
            self._value = np.array([value])
            self._value.reshape(shape)
        
        elif isinstance(value, list):
            value = np.array(value)
            assert value.shape == shape, "Shape of value must be the same as the shape of the parameter"
            self._value = value
        
        elif isinstance(value, np.ndarray):
            assert value.shape == shape, "Shape of value must be the same as the shape of the parameter"
            self._value = value
        
        else:
            raise ValueError("Value must be a scalar or numpy array")

        if self._min_value is not None:
            self._value = np.maximum(self._value, self._min_value)
        if self._max_value is not None:
            self._value = np.minimum(self._value, self._max_value)

    @property
    def shape(self):
        """
        Returns: The shape of the table of parameters.
        """
        return self._value.shape
    
    @property
    def size(self):
        """
        Returns: The size of the table of parameters.
        """
        return self._value.size 

In [ ]:
show_doc(Parameter, title_level=2)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/utils.py#L28){target="_blank" style="float:right; font-size:smaller"}

## Parameter

>      Parameter (value:int|float|list[int]|list[float]|numpy.ndarray,
>                 min_value:int|float|list[int]|list[float]|numpy.ndarray=None,
>                 max_value:int|float|list[int]|list[float]|numpy.ndarray=None,
>                 shape:tuple[int]=(1,))

*Simple class to handle parameters in the environment. The advantage of this class is that it can be
used to set parameters that may change over time and accessed by multiple objects such as the 
environment, agent or dataloaders.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| value | int \| float \| list[int] \| list[float] \| numpy.ndarray |  | the value of the parameter |
| min_value | int \| float \| list[int] \| list[float] \| numpy.ndarray | None | the minimum value of the parameter |
| max_value | int \| float \| list[int] \| list[float] \| numpy.ndarray | None | the maximum value of the parameter |
| shape | tuple | (1,) | the shape of the parameter |

In [ ]:
show_doc(Parameter.__call__)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/utils.py#L47){target="_blank" style="float:right; font-size:smaller"}

### Parameter.__call__

>      Parameter.__call__ ()

*Returns the parameter value. Alternative to get_value().*

In [ ]:
show_doc(Parameter.get_value)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/utils.py#L54){target="_blank" style="float:right; font-size:smaller"}

### Parameter.get_value

>      Parameter.get_value ()

*Returns the parameter value.*

In [ ]:
show_doc(Parameter.set_value)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/utils.py#L63){target="_blank" style="float:right; font-size:smaller"}

### Parameter.set_value

>      Parameter.set_value
>                           (value:Union[int,float,List[int],List[float],numpy.n
>                           darray], shape:Tuple[int]=(1,))

*Set the value of the parameter. The value can be a scalar, list or numpy array. The shape of the value
must be the same as the shape of the parameter.

All values, including scalars, are converted to numpy arrays. If the value is a scalar, it is reshaped
to the shape of the parameter. If the value is a list or numpy array, it must have the same shape as the
parameter. If the value is a scalar, it is reshaped to the shape of the parameter.

Optionally, the value can be clipped to a minimum and maximum value.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| value | Union |  | the value of the parameter |
| shape | Tuple | (1,) | the shape of the parameter |

In [ ]:
show_doc(Parameter.shape)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/utils.py#L100){target="_blank" style="float:right; font-size:smaller"}

### Parameter.shape

>      Parameter.shape ()

*Returns: The shape of the table of parameters.*

In [ ]:
show_doc(Parameter.size)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/utils.py#L107){target="_blank" style="float:right; font-size:smaller"}

### Parameter.size

>      Parameter.size ()

*Returns: The size of the table of parameters.*

Example usage for the `Parameter` class:

In [ ]:
overage_cost = Parameter(1) # integer
underage_cost = Parameter(2.13) # float
ordering_cost = Parameter([2]*5, shape=(5,)) # list
holding_cost = Parameter(np.array([0,1]), shape=(2,)) # numpy array

print(overage_cost.get_value())
print(underage_cost.get_value())
print(ordering_cost.get_value())
print(holding_cost.get_value())

[1]
[2.13]
[2 2 2 2 2]
[0 1]


In [ ]:
#| export

class MDPInfo():
    """
    This class is used to store the information of the environment.
    It is based on MushroomRL (https://github.com/MushroomRL). It can be accessed by 
    agents that need the information of the environment, such as the state and action spaces.
    
    Key difference with MushroomRL is that the state and action spaces are gymnasium spaces.
    """
    
    def __init__(self,
                observation_space: Space,
                action_space: Space,  
                gamma: float,
                horizon: int,
                dt: float = 1e-1,
                backend: Literal['numpy'] = 'numpy'  # Currently only numpy is supported
            ) -> None: 

        self.observation_space = observation_space
        self.action_space = action_space
        self.gamma = gamma
        self.horizon = horizon
        self.dt = dt
        self.backend = backend

    @property
    def size(self):
        """
        Returns: The sum of the number of discrete states and discrete actions. Only works for discrete spaces.

        """
        return self.observation_space.size + self.action_space.size

    @property
    def shape(self):
        """
        Returns: The concatenation of the shape tuple of the state and action spaces.

        """
        return self.observation_space.shape + self.action_space.shape

In [ ]:
show_doc(MDPInfo, title_level=2)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/utils.py#L114){target="_blank" style="float:right; font-size:smaller"}

## MDPInfo

>      MDPInfo (observation_space:gymnasium.spaces.space.Space,
>               action_space:gymnasium.spaces.space.Space, gamma:float,
>               horizon:int, dt:float=0.1, backend:Literal['numpy']='numpy')

*This class is used to store the information of the environment.
It is based on MushroomRL (https://github.com/MushroomRL). It can be accessed by 
agents that need the information of the environment, such as the state and action spaces.

Key difference with MushroomRL is that the state and action spaces are gymnasium spaces.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| observation_space | Space |  |  |
| action_space | Space |  |  |
| gamma | float |  |  |
| horizon | int |  |  |
| dt | float | 0.1 |  |
| backend | Literal | numpy | Currently only numpy is supported |
| **Returns** | **None** |  |  |

In [ ]:
show_doc(MDPInfo.size)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/utils.py#L140){target="_blank" style="float:right; font-size:smaller"}

### MDPInfo.size

>      MDPInfo.size ()

*Returns: The sum of the number of discrete states and discrete actions. Only works for discrete spaces.*

In [ ]:
show_doc(MDPInfo.shape)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/utils.py#L148){target="_blank" style="float:right; font-size:smaller"}

### MDPInfo.shape

>      MDPInfo.shape ()

*Returns: The concatenation of the shape tuple of the state and action spaces.*

In [ ]:
#| export

class DatasetWrapper(Dataset):
    """
    This class is used to wrap a Pytorch Dataset around the ddopnew dataloader
    to enable the usage of the Pytorch Dataloader during training. This way,
    agents that are trained using Pytorch without interacting with the environment
    can directly train on the data generated by the dataloader.
    
    """

    def __init__(self, 
            dataloader: BaseDataLoader # Any dataloader that inherits from BaseDataLoader
            ):
        """


        """
        self.dataloader = dataloader
    
    def __getitem__(self, idx):
        """
        Get the item at the provided idx.

        """

        # create tuple of items
        return self.dataloader[idx]


    def __len__(self):

        """

        Returns the length of the dataset. Depends on the state of
        the dataloader (train, val, test).

        """
        
        if self.dataloader.dataset_type == 'train':
            return self.dataloader.len_train
        elif self.dataloader.dataset_type == 'val':
            return self.dataloader.len_val
        elif self.dataloader.dataset_type == 'test':
            return self.dataloader.len_test
        else:
            raise ValueError("Dataset type must be either 'train', 'val' or 'test'")

In [ ]:
show_doc(DatasetWrapper, title_level=2)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/utils.py#L156){target="_blank" style="float:right; font-size:smaller"}

## DatasetWrapper

>      DatasetWrapper (dataloader:ddopnew.dataloaders.base.BaseDataLoader)

*This class is used to wrap a Pytorch Dataset around the ddopnew dataloader
to enable the usage of the Pytorch Dataloader during training. This way,
agents that are trained using Pytorch without interacting with the environment
can directly train on the data generated by the dataloader.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| dataloader | BaseDataLoader | Any dataloader that inherits from BaseDataLoader |

In [ ]:
show_doc(DatasetWrapper.__getitem__)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/utils.py#L174){target="_blank" style="float:right; font-size:smaller"}

### DatasetWrapper.__getitem__

>      DatasetWrapper.__getitem__ (idx)

*Get the item at the provided idx.*

In [ ]:
show_doc(DatasetWrapper.__len__)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/utils.py#L184){target="_blank" style="float:right; font-size:smaller"}

### DatasetWrapper.__len__

>      DatasetWrapper.__len__ ()

*Returns the length of the dataset. Depends on the state of
the dataloader (train, val, test).*

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()